In [1]:
import numpy as np

# Lecture et prétraitement

In [2]:
def traiter(url):
    phrases = []
    classes = []
    with open(url, mode='r', encoding='utf-8') as f:
        for ligne in f:
            parties = ligne.split('\t')
            phrases.append(parties[3])
            classes.append(int(parties[1]))
    return phrases, classes

In [3]:
X_train, Y_train = traiter('./cola_public/tokenized/in_domain_train.tsv')

X_train[:3], Y_train[:3]

(["our friends wo n't buy this analysis , let alone the next one we propose .\n",
  "one more pseudo generalization and i 'm giving up .\n",
  "one more pseudo generalization or i 'm giving up .\n"],
 [1, 1, 1])

In [4]:
X_dev, Y_dev = traiter('./cola_public/tokenized/in_domain_dev.tsv')

X_dev[:3], Y_dev[:3]

(['the sailors rode the breeze clear of the rocks .\n',
  'the weights made the rope stretch over the pulley .\n',
  'the mechanical doll wriggled itself loose .\n'],
 [1, 1, 1])

In [5]:
from nltk.lm import MLE, Laplace, KneserNeyInterpolated
from nltk.lm import Vocabulary
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline


class NGramme:
    def __init__(self, n=2, cutoff=2):
        self.n = n
        self.kn = KneserNeyInterpolated(n, vocabulary=Vocabulary(unk_cutoff=cutoff))
    
    def preprocess(self, X): 
        sent_words = []
        for sent in X:
            sent_words.append(list(pad_both_ends(sent.split(), self.n)))
        return sent_words

    def entrainer(self, X):
        sent_words = self.preprocess(X)
        train, vocab = padded_everygram_pipeline(self.n, sent_words)
        vocab_l = list(vocab)
        train_l = []
        for t in train:
            train_l.append(list(t))
        self.kn.fit(train_l, vocab_l)
        
    def noter_phrase(self, phrase):
        notes = []
        print('notation de la phrase :', phrase)
        for i in range(len(phrase) - self.n+1):
            notes.append(self.kn.score(phrase[i+self.n-1], phrase[i:i+self.n-1]))
            
        return notes
    
    def noter(self, X):
        notes = []
        sent_words = self.preprocess(X)
        for phrase in sent_words:
            notes.append(self.noter_phrase(phrase))
        return notes
        

ngram = NGramme(3)

ngram.entrainer(X_train)

ngram.noter(X_train[:3])

notation de la phrase : ['<s>', '<s>', 'our', 'friends', 'wo', "n't", 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'one', 'more', 'pseudo', 'generalization', 'and', 'i', "'m", 'giving', 'up', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'one', 'more', 'pseudo', 'generalization', 'or', 'i', "'m", 'giving', 'up', '.', '</s>', '</s>']


[[0.00038959067363963197,
  0.09407374951283498,
  0.18207931919052078,
  0.9993337676615498,
  0.027084209034642058,
  0.4525122154810864,
  0.900593680417298,
  0.9191117309336699,
  0.4503031857767192,
  0.46125071256973005,
  0.9302447168044302,
  0.90014885702644,
  0.23250787219892233,
  0.903029768933511,
  0.45184982859547174,
  0.19174770759528778,
  0.9979187272878222,
  0.9999999915199806],
 [0.00046768632561359027,
  0.15893702477846783,
  0.950328578608404,
  0.9950002035913514,
  0.4951160470703205,
  0.9028548077889896,
  0.06306726241546798,
  0.03491282361202489,
  0.961256514923246,
  0.9619886460530087,
  0.9999847296875257,
  0.9999999915199806],
 [0.00046768632561359027,
  0.15893702477846783,
  0.950328578608404,
  0.9950002035913514,
  0.4950171016735209,
  0.9058259375381734,
  0.30324030205173574,
  0.03491282361202489,
  0.961256514923246,
  0.9619886460530087,
  0.9999847296875257,
  0.9999999915199806]]

# Modèle neuronal

In [28]:
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

class MLNR: # Modèle de langage neuronal (récurrent)
    def __init__(self):
        # define model
        self.modele = Sequential()

        # self.modele.summary()
        self.tokenizer = Tokenizer(oov_token='<UNK>')
        
    def preprocess(self, X, train=False):
        sentencesPad2 = []
        for sentence in X:
            new_sentence = "<s> " + sentence + " </s>"
            sentencesPad2.append(new_sentence)
        if train:
            self.tokenizer.fit_on_texts(sentencesPad2)
        
        total_words2 = len(self.tokenizer.word_index) + 1

        # each sentence is encoded as lists of 2 to its length
        sequences = []
        for line in sentencesPad2:
            encoded = self.tokenizer.texts_to_sequences([line])[0]
            for i in range(1, len(encoded)):
                sequence = encoded[:i+1]
                sequences.append(sequence)
        if train:
            self.max_length = max([len(seq) for seq in sequences])
        #add 
        sequences = pad_sequences(sequences, maxlen=self.max_length, padding='pre')
        sequences = np.array(sequences)

        return sequences[:,:-1], to_categorical(sequences[:,-1], num_classes=total_words2)
    
    def entrainer(self, X, epochs=10):
        X2, Y2 = self.preprocess(X, train=True)
        self.modele.add(Embedding(Y2.shape[1], 10, input_length=X2.shape[1]))
        self.modele.add(LSTM(50))
        self.modele.add(Dense(Y2.shape[1], activation='softmax'))
        self.modele.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.modele.fit(X2, Y2, epochs=epochs, verbose=2)
        
    def noter(self, X):
        
        X2, Y2 = self.preprocess(X)
        return self.modele.predict(X2)
    
nr = MLNR()

nr.entrainer(X_train)

nr.noter(X_train[:3])

Epoch 1/10
2353/2353 - 24s - loss: 6.1240 - accuracy: 0.1256 - 24s/epoch - 10ms/step
Epoch 2/10
2353/2353 - 23s - loss: 5.6061 - accuracy: 0.1646 - 23s/epoch - 10ms/step
Epoch 3/10
2353/2353 - 23s - loss: 5.2786 - accuracy: 0.1811 - 23s/epoch - 10ms/step
Epoch 4/10
2353/2353 - 23s - loss: 5.0861 - accuracy: 0.1890 - 23s/epoch - 10ms/step
Epoch 5/10
2353/2353 - 23s - loss: 4.9391 - accuracy: 0.1954 - 23s/epoch - 10ms/step
Epoch 6/10
2353/2353 - 23s - loss: 4.7967 - accuracy: 0.2050 - 23s/epoch - 10ms/step
Epoch 7/10
2353/2353 - 23s - loss: 4.6600 - accuracy: 0.2156 - 23s/epoch - 10ms/step
Epoch 8/10
2353/2353 - 23s - loss: 4.5338 - accuracy: 0.2244 - 23s/epoch - 10ms/step
Epoch 9/10
2353/2353 - 23s - loss: 4.4200 - accuracy: 0.2312 - 23s/epoch - 10ms/step
Epoch 10/10
2353/2353 - 23s - loss: 4.3143 - accuracy: 0.2378 - 23s/epoch - 10ms/step


array([[1.19700672e-09, 1.17713472e-09, 1.79037059e-04, ...,
        4.27341765e-05, 1.35178925e-05, 2.17085486e-07],
       [2.33722686e-09, 2.32413933e-09, 1.73985783e-04, ...,
        7.68890658e-08, 8.16440462e-08, 3.04848508e-10],
       [5.86778448e-08, 6.16546956e-08, 2.33251345e-03, ...,
        1.47645063e-10, 9.03340458e-11, 1.39899828e-10],
       ...,
       [1.50669557e-08, 1.52453747e-08, 2.82721408e-02, ...,
        3.79719811e-09, 7.09612946e-09, 1.11942015e-07],
       [1.26457469e-10, 1.11122118e-10, 3.17987055e-01, ...,
        2.24535017e-13, 4.02889050e-12, 8.13751939e-14],
       [2.26883651e-11, 1.98315375e-11, 6.67566717e-01, ...,
        3.89506396e-12, 2.99660852e-10, 1.19044319e-15]], dtype=float32)

In [12]:
Ps_train = ngram.noter(X_train)

notation de la phrase : ['<s>', '<s>', 'our', 'friends', 'wo', "n't", 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'one', 'more', 'pseudo', 'generalization', 'and', 'i', "'m", 'giving', 'up', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'one', 'more', 'pseudo', 'generalization', 'or', 'i', "'m", 'giving', 'up', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'we', 'study', 'verbs', ',', 'the', 'crazier', 'they', 'get', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'day', 'by', 'day', 'the', 'facts', 'are', 'getting', 'murkier', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', "'ll", 'fix', 'you', 'a', 'drink', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'fred', 'watered', 'the', 'plants', 'flat', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'bill', 'coughed', 'his', 'way', 'out', 'of'

notation de la phrase : ['<s>', '<s>', 'the', 'weights', 'stretched', 'the', 'rope', 'over', 'the', 'pulley', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'weights', 'kept', 'the', 'rope', 'stretched', 'over', 'the', 'pulley', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'sam', 'cut', 'himself', 'free', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'sam', 'got', 'free', 'by', 'cutting', 'his', 'finger', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'bill', 'cried', 'himself', 'to', 'sleep', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'bill', 'cried', 'sue', 'to', 'sleep', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'bill', 'squeezed', 'himself', 'through', 'the', 'hole', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'bill', 'sang', 'himself', 'to', 'sleep', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'bill', 'squeezed', 'the', 'puppet', 'through', 'the', 'hole'

notation de la phrase : ['<s>', '<s>', 'when', 'john', 'eats', 'more', ',', 'keep', 'your', 'mouth', 'shut', 'tighter', ',', 'ok', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'as', 'john', 'eats', 'more', ',', 'keep', 'your', 'mouth', 'shut', 'tighter', ',', 'ok', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'keep', 'your', 'mouth', 'shut', 'tighter', ',', 'the', 'more', 'john', 'eats', ',', 'ok', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'everyone', 'keep', 'your', 'mouth', 'shut', 'tighter', ',', 'the', 'more', 'john', 'eats', ',', 'ok', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'can', 'well', 'imagine', 'the', 'more', 'him', 'eating', ',', 'the', 'fatter', 'him', 'getting', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'bill', 'can', 'well', 'imagine', 'getting', 'fat', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'bill', 'can', 'well', 'imagine', 'the', 'more', 'he', 'eats', ',', 't

notation de la phrase : ['<s>', '<s>', 'the', 'more', 'books', 'i', 'ask', 'to', 'whom', 'he', 'will', 'give', ',', 'the', 'more', 'he', 'reads', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'people', 'i', 'ask', 'what', 'he', 'will', 'give', 'to', 'the', 'more', 'he', 'reads', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'carefully', 'he', 'words', 'the', 'letter', 'the', 'safer', 'he', "'ll", 'be', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'carefully', 'he', 'knows', 'a', 'man', 'that', 'worded', 'the', 'letter', 'the', 'safer', 'he', "'ll", 'be', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'geniuses', 'john', 'meets', ',', 'the', 'angrier', 'he', 'gets', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'john', 'meets', 'geniuses', ',', 'the', 'angrier', 'he', 'gets', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'm

notation de la phrase : ['<s>', '<s>', 'so', 'much', 'did', 'he', 'eat', 'that', 'he', 'got', 'sick', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'you', 'eat', ',', 'the', 'more', 'you', 'want', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'you', 'eat', 'the', 'more', ',', 'the', 'more', 'you', 'want', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'you', 'eat', ',', 'you', 'want', 'the', 'more', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'wonder', 'you', 'ate', 'how', 'much', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'wonder', 'to', 'how', 'many', 'people', 'bill', 'talks', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'longer', 'he', 'has', 'to', 'wait', ',', 'the', 'angrier', 'john', 'gets', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'if', 'he', 'has', 'to', 'wait', ',', 'john', 'gets', 'angry', '.', '</s>', '</s>']
notation de 

...

notation de la phrase : ['<s>', '<s>', 'john', 'saw', 'stephan', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'destroyed', 'there', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'what', 'was', 'euclid', 'interested', 'in', 'plato', "'s", 'description', 'of', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'like', 'anson', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'dragons', 'simply', 'all', 'died', 'out', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'gilgamesh', 'did', 'not', 'fly', 'the', 'dragon', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'which', 'goddess', 'might', 'help', 'us', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'humans', 'love', 'to', 'eat', 'pigs', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'which', 'poem', 'about', 'achilles', 'did', 'homer', 'recite', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'boys', 'should', 'al

notation de la phrase : ['<s>', '<s>', 'poseidon', 'appears', 'to', 'have', 'turned', 'out', 'to', 'have', 'left', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'gilgamesh', 'has', 'been', 'not', 'reading', 'the', 'cuneiform', 'tablets', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'danced', 'extremely', ',', 'anson', 'frantically', 'at', 'trade', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'aphrodite', 'wanted', 'to', 'live', 'and', 'ishtar', 'tried', 'to', 'do', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'kicked', 'yourself', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'how', 'fond', 'of', 'esther', 'is', 'agamemnon', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'ron', 'heard', 'a', 'discussion', 'in', 'the', 'foyer', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'my', 'mother', 'hated', 'myself', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'students', 'demonstrated', 'the', 

notation de la phrase : ['<s>', '<s>', 'i', "'d", 'planned', 'to', 'have', 'finished', ',', 'and', 'finished', 'i', 'have', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'ron', 'expected', 'the', 'sack', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'that', 'i', 'am', 'here', 'proves', 'that', 'i', 'care', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'she', 'tried', 'to', 'may', 'leave', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'gilgamesh', 'misses', 'aphrodite', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'who', 'seemed', 'had', 'poisoned', 'jason', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'that', 'plato', 'loved', 'aster', 'seemed', 'to', 'be', 'known', 'by', 'everyone', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'when', 'dining', 'with', 'evil', 'crocodiles', ',', 'it', 'is', 'advisable', 'to', 'wear', 'armour', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'benjamin', 's

notation de la phrase : ['<s>', '<s>', 'humans', 'love', 'to', 'eat', 'owners', 'of', 'pigs', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'have', 'sent', '0', 'letter', 'to', 'environmental', 'heath', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'what', 'jason', 'asked', 'whether', 'was', 'the', 'potion', 'was', 'ready', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'those', 'pigs', 'love', 'truffles', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'we', 'all', 'thought', 'he', 'to', 'be', 'unhappy', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', "'d", 'planned', 'to', 'have', 'finished', 'by', 'now', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'has', 'the', 'potion', 'not', 'worked', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'what', 'i', 'love', 'is', 'toast', 'and', 'sun', 'dried', 'tomatoes', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'mary', 'ran', '.', '</s>', '</s>']
notatio

notation de la phrase : ['<s>', '<s>', 'gilgamesh', 'did', "n't", 'leave', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'genie', 'intoned', 'that', 'she', 'was', 'tired', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'look', 'at', 'all', 'these', 'books', '.', 'which', 'book', 'would', 'you', 'like', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'there', 'were', 'killed', 'three', 'men', 'by', 'the', 'assassin', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'peter', 'is', 'those', 'pigs', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'do', "n't", 'remember', 'what', 'i', 'said', 'all', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'pig', 'grunts', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'poison', 'was', 'neutralised', 'was', 'claimed', 'that', 'by', 'everyone', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'people', 'are', 'stupid', '</s>', '</s>']
notation de la p

In [14]:
Ps_train[:3]

[[0.00038959067363963197,
  0.09407374951283498,
  0.18207931919052078,
  0.9993337676615498,
  0.027084209034642058,
  0.4525122154810864,
  0.900593680417298,
  0.9191117309336699,
  0.4503031857767192,
  0.46125071256973005,
  0.9302447168044302,
  0.90014885702644,
  0.23250787219892233,
  0.903029768933511,
  0.45184982859547174,
  0.19174770759528778,
  0.9979187272878222,
  0.9999999915199806],
 [0.00046768632561359027,
  0.15893702477846783,
  0.950328578608404,
  0.9950002035913514,
  0.4951160470703205,
  0.9028548077889896,
  0.06306726241546798,
  0.03491282361202489,
  0.961256514923246,
  0.9619886460530087,
  0.9999847296875257,
  0.9999999915199806],
 [0.00046768632561359027,
  0.15893702477846783,
  0.950328578608404,
  0.9950002035913514,
  0.4950171016735209,
  0.9058259375381734,
  0.30324030205173574,
  0.03491282361202489,
  0.961256514923246,
  0.9619886460530087,
  0.9999847296875257,
  0.9999999915199806]]

In [31]:
class AcceptSepBasique:
    def __init__(self, modele):
        self.modele = modele
    
    def estimer_log_proba(self, SP):
        res = []
        for sp in SP:
            res.append(np.log(sp).sum())
        return np.array(res)
                
    def entrainer(self, X, Y):
        idx = np.array(Y) == 1
        pos_min = self.estimer_log_proba(self.modele.noter(X[idx])).min()
        neg_max = self.estimer_log_proba(self.modele.noter(X[~idx])).max()
        
        self.sep = neg_max + (pos_min-neg_max)/2
    
    def _entrainer(self, Ps, Y):
        idx = np.array(Y) == 1
        npPs = self.estimer_log_proba(Ps)
        pos_min = npPs[idx].min()
        neg_max = (npPs[~idx]).max()
        
        self.sep = neg_max + (pos_min-neg_max)/2
        
    def estimer(self, X):
        return (np.array(self.estimer_log_proba(self.modele.noter(X))) >= self.sep).astype(int)
        

asb = AcceptSepBasique(ngram)

asb._entrainer(Ps_train, Y_train)

Y_est = asb.estimer(X_dev)

from sklearn.metrics import classification_report

print(classification_report(Y_dev, Y_est))

notation de la phrase : ['<s>', '<s>', 'the', 'sailors', 'rode', 'the', 'breeze', 'clear', 'of', 'the', 'rocks', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'weights', 'made', 'the', 'rope', 'stretch', 'over', 'the', 'pulley', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'mechanical', 'doll', 'wriggled', 'itself', 'loose', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'if', 'you', 'had', 'eaten', 'more', ',', 'you', 'would', 'want', 'less', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'as', 'you', 'eat', 'the', 'most', ',', 'you', 'want', 'the', 'least', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'you', 'would', 'want', ',', 'the', 'less', 'you', 'would', 'eat', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'demand', 'that', 'the', 'more', 'john', 'eat', ',', 'the', 'more', 'he', 'pays', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'mary', 'listens

notation de la phrase : ['<s>', '<s>', 'a', 'hundred', 'men', 'surrounded', 'the', 'fort', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'we', 'elected', 'me', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'which', 'report', 'that', 'john', 'was', 'incompetent', 'did', 'he', 'submit', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'mary', 'has', 'always', 'preferred', 'lemons', 'to', 'limes', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'he', 'let', 'the', 'cats', 'which', 'were', 'whining', 'out', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'what', 'did', 'bill', 'buy', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'mary', 'saw', 'the', 'boy', 'walking', 'toward', 'the', 'railroad', 'station', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'a', 'proof', 'that', 'the', 'claim', 'had', 'been', 'made', 'was', 'given', 'that', 'john', 'had', 'lied', '.', '</s>', '</s>']
notation de la phrase

notation de la phrase : ['<s>', '<s>', 'no', 'writer', ',', 'nor', 'any', 'playwright', ',', 'meets', 'in', 'vienna', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'that', 'you', 'will', 'marry', 'any', 'student', 'is', 'not', 'certain', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'felicia', 'kicked', 'the', 'ball', 'off', 'the', 'bench', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'sent', 'the', 'package', 'halfway', 'around', 'the', 'world', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'sam', 'gave', 'the', 'ball', 'out', 'of', 'the', 'basket', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'sam', 'offered', 'the', 'ball', 'out', 'of', 'the', 'basket', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'park', 'square', 'has', 'a', 'festive', 'air', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'worker', 'will', 'have', 'a', 'job', '.', '</s>', '</s>']
notation de la phrase : 

notation de la phrase : ['<s>', '<s>', 'the', 'witch', 'poisoned', 'the', 'children', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'a', 'grandfather', 'clock', 'ticked', 'in', 'the', 'hallway', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'squeaked', 'the', 'door', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'over', 'the', 'fire', 'there', 'bubbled', 'a', 'fragrant', 'stew', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'soaring', 'temperatures', 'are', 'predicted', 'for', 'this', 'weekend', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'fort', 'fluttered', 'with', 'many', 'flags', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'voices', 'echoed', 'in', 'the', 'hall', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'stream', 'twists', 'through', 'the', 'valley', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'there', 'presented', 'itself', 'a', 'wond

notation de la phrase : ['<s>', '<s>', 'there', 'is', 'believed', 'to', 'be', 'sheep', 'in', 'the', 'park', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'hope', 'to', 'would', 'study', 'in', 'france', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'john', 'can', 'kick', 'the', 'ball', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'john', 'will', 'rain', 'tomorrow', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'we', 'expect', 'there', 'to', 'will', 'rain', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'john', 'was', 'found', 'in', 'the', 'office', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'did', 'the', 'child', 'be', 'in', 'the', 'school', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'it', 'did', 'not', 'rain', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'john', 'wants', 'not', 'to', 'leave', 'the', 'town', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>

notation de la phrase : ['<s>', '<s>', 'gwen', 'hit', 'the', 'baseball', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'hit', 'that', 'you', 'knew', 'the', 'answer', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', "'ve", 'lost', 'my', 'wallet', 'or', 'i', "'ve", 'lost', 'my', 'mind', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'it', 'was', 'a', 'brand', 'new', 'car', 'that', 'he', 'bought', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'he', 'likes', 'cookies', 'and', 'he', 'hates', 'crumb', 'cake', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'they', 'chased', 'the', 'man', 'with', 'the', 'car', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'did', "n't", 'have', 'a', 'red', 'cent', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'book', 'of', 'poems', 'and', 'from', 'blackwell', 'takes', 'a', 'very', 'long', 'time', 'to', 'read', '.', '</s>', '</s>']
notation de la ph

notation de la phrase : ['<s>', '<s>', 'magnus', 'went', 'to', 'ireland', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'who', 'did', 'john', 'send', 'the', 'book', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'idea', 'dismayed', 'the', 'prime', 'minister', 'that', 'the', 'dome', 'was', 'dull', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'came', 'right', 'in', 'he', 'did', 'without', 'so', 'much', 'as', 'a', 'knock', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'harriet', 'admired', 'mr', 'knightley', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'book', 'is', 'boring', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'ethel', 'wishes', 'to', 'ask', 'you', 'some', 'awkward', 'questions', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'emma', 'made', 'harriet', 'some', 'food', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'window', 'was', 'broken', 'with',

notation de la phrase : ['<s>', '<s>', 'it', 'is', 'not', 'true', 'that', 'i', 'might', 'be', 'going', 'to', 'the', 'party', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'kick', 'you', '!', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'jason', 'persuaded', 'the', 'doctor', 'to', 'treat', 'medea', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'who', 'did', 'you', 'get', 'an', 'accurate', 'description', 'of', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'peter', 'we', 'all', 'like', 'was', 'at', 'the', 'party', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'government', "'s", 'imposition', 'of', 'a', 'fine', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'did', 'athena', 'help', 'us', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'you', 'wanted', 'to', 'meet', 'the', 'man', 'i', 'saw', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'anson', 'demonized', 'david', 'every', 'day',

In [32]:
print(classification_report(Y_dev, Y_est))

              precision    recall  f1-score   support

           0       0.30      0.53      0.38       162
           1       0.68      0.45      0.54       365

    accuracy                           0.47       527
   macro avg       0.49      0.49      0.46       527
weighted avg       0.56      0.47      0.49       527



In [ ]:
from sklearn.linear_model import LogisticRegression

class AcceptSepReg:
    def __init__(self, modele):
        self.modele = modele
        self.classif = LogisticRegression(penalty='none', max_iter=10)
    
    def __estimer_proba(self, SP):
        res = []
        for sp in SP:
            spl = np.log(sp)
            res.append([spl.sum()])
        return np.array(res)
                
    def entrainer(self, X, Y):
        SP = self.modele.noter(X)
        XX = self.__estimer_proba(SP)
        self.classif.fit(XX, Y)
        
    def _entrainer(self, SP, Y):
        XX = self.__estimer_proba(SP)
        self.classif.fit(XX, Y)
    
    def estimer(self, X):
        SP = self.modele.noter(X)
        XX = self.__estimer_proba(SP)
        return self.classif.predict(XX)
        

asr = AcceptSepReg(ngram)

asr._entrainer(Ps_train, Y_train)

Y_est2 = asr.estimer(X_dev)

from sklearn.metrics import classification_report

print(classification_report(Y_dev, Y_est2))

/home/mc02-10/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


notation de la phrase : ['<s>', '<s>', 'the', 'sailors', 'rode', 'the', 'breeze', 'clear', 'of', 'the', 'rocks', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'weights', 'made', 'the', 'rope', 'stretch', 'over', 'the', 'pulley', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'mechanical', 'doll', 'wriggled', 'itself', 'loose', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'if', 'you', 'had', 'eaten', 'more', ',', 'you', 'would', 'want', 'less', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'as', 'you', 'eat', 'the', 'most', ',', 'you', 'want', 'the', 'least', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'more', 'you', 'would', 'want', ',', 'the', 'less', 'you', 'would', 'eat', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'demand', 'that', 'the', 'more', 'john', 'eat', ',', 'the', 'more', 'he', 'pays', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'mary', 'listens

notation de la phrase : ['<s>', '<s>', 'rusty', 'talked', 'about', 'himself', 'only', 'after', 'mary', 'did', 'talk', 'about', 'him', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'fred', 'talked', 'about', 'everything', 'before', 'rusty', 'did', 'talk', 'about', 'something', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'john', 'often', 'meets', 'mary', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'problem', 'perceives', 'easily', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'a', 'hundred', 'men', 'surrounded', 'the', 'fort', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'we', 'elected', 'me', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'which', 'report', 'that', 'john', 'was', 'incompetent', 'did', 'he', 'submit', '?', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'mary', 'has', 'always', 'preferred', 'lemons', 'to', 'limes', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s

notation de la phrase : ['<s>', '<s>', 'nobody', 'who', 'hates', 'to', 'eat', 'anything', 'should', 'work', 'in', 'a', 'delicatessen', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'everybody', 'around', 'here', 'who', 'ever', 'buys', 'anything', 'on', 'credit', 'talks', 'in', 'his', 'sleep', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'ca', "n't", 'remember', 'the', 'name', 'of', 'somebody', 'who', 'had', 'misgivings', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'no', 'writer', ',', 'and', 'no', 'playwright', ',', 'meets', 'in', 'vienna', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'no', 'writer', ',', 'nor', 'any', 'playwright', ',', 'meets', 'in', 'vienna', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'that', 'you', 'will', 'marry', 'any', 'student', 'is', 'not', 'certain', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'felicia', 'kicked', 'the', 'ball', 'off', 'the', 'bench', '.', '</s>'

notation de la phrase : ['<s>', '<s>', 'i', 'shaped', 'a', 'loaf', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'children', 'amused', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'susan', 'whispered', 'the', 'news', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'susan', 'whispered', 'at', 'rachel', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'ellen', 'said', 'that', 'melons', 'were', 'selling', 'well', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'ellen', 'said', 'about', 'the', 'present', 'conditions', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'ellen', 'warned', 'helen', 'against', 'skating', 'on', 'thin', 'ice', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'cynthia', 'nibbled', 'on', 'the', 'carrot', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'cynthia', 'chewed', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'paul', 'laughed', 'at', 'mary',

notation de la phrase : ['<s>', '<s>', 'i', 'drank', 'some', 'of', 'water', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'it', 'is', 'a', 'golden', 'hair', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'boy', 'in', 'the', 'doorway', 'waved', 'to', 'his', 'father', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'that', 'dog', 'is', 'so', 'ferocious', ',', 'it', 'even', 'tried', 'to', 'bite', 'itself', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'he', 'washed', 'yourself', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'john', 'is', 'easy', 'to', 'please', 'kim', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'there', 'seemed', 'to', 'be', 'intelligent', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'john', 'tried', 'to', 'please', 'stephen', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'we', 'believed', 'john', 'to', 'be', 'a', 'fountain', 'in', 'the', 'park', '.', '</

notation de la phrase : ['<s>', '<s>', 'with', 'no', 'job', 'would', 'john', 'be', 'happy', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'have', 'much', 'of', 'the', 'manuscript', 'left', 'to', 'type', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'he', "'s", 'a', 'more', 'reliable', 'man', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'any', 'trouble', 'is', 'what', 'i', 'do', "n't", 'want', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'they', 'may', 'grow', 'as', 'high', 'as', 'bamboo', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'some', 'of', 'them', 'made', 'as', 'many', 'errors', 'as', '20', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'sally', 'kissed', 'himself', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'drew', 'believes', 'i', 'think', 'rosie', 'loves', 'magazine', 'ads', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'dave', ',', 'dan', ',', 'erin', ','

notation de la phrase : ['<s>', '<s>', 'the', 'kennel', 'which', 'mary', 'made', 'and', 'fido', 'sleeps', 'has', 'been', 'stolen', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'tom', 'said', 'he', 'would', 'and', 'bill', 'actually', 'did', 'eat', 'a', 'raw', 'eggplant', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'wealthy', 'young', 'man', 'bought', 'that', 'piano', 'for', 'his', 'secret', 'fiancée', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'the', 'dog', 'stole', 'the', 'turkey', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'a', 'tall', 'building', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'this', 'building', 'is', 'tall', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'i', 'like', 'the', 'book', 'which', 'you', 'gave', 'me', '.', '</s>', '</s>']
notation de la phrase : ['<s>', '<s>', 'captain', 'wentworth', 'wrote', 'a', 'letter', 'to', 'anne', 'elliott', '.', '</s>', '</s>']
notatio